太多特徵跑不動 coxPH 即使有標準化

In [7]:
import pandas as pd
import numpy as np

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.pipeline import  make_pipeline
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
import joblib

In [4]:
def c_index(clf, X, y_struct):
    return concordance_index_censored(y_struct['Status'],y_struct['Survival'],clf.predict(X))[0]

In [ ]:
def cx_pipelines(pipelines, X, y_struct, cv=5, scoring=c_index):
    scores = dict()

    for model, pipeline in pipelines.items():
        print(f"{model}: Cross validation...")
        score = cross_val_score(pipeline, X, y_struct, cv=cv, scoring=scoring)
        scores[model] = score
        print(f"Score:{score}")
        print("Done.")
        print("="*8)

    return scores

In [2]:
training_df = pd.read_csv("../../data/train_test/training.csv")

In [3]:
X_train = training_df.drop(columns=['survival_status', 'survival_time', 'recurrence', 'metastasis'])
y_train_struct = training_df[['survival_status', 'survival_time']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

In [6]:
exps = {
    'pvalue': list(pd.read_csv("../../data/selected_features/p-value-recur-related-train.csv").columns),
    'corr': list(pd.read_csv("../../data/selected_features/corr-recur-related-train.csv").columns),
    'rf': list(pd.read_csv("../../data/selected_features/rf-recur-related-train.csv").columns),
    'p_corr': list(pd.read_csv("../../data/selected_features/cor_p-recur-related-train.csv").columns),
}

In [8]:
cox_pipeline = make_pipeline(StandardScaler(), CoxPHSurvivalAnalysis())

In [10]:
models = dict()
for exp, features in exps.items():
    print(f"Exp: {exp}")
    X_train_Selected = X_train[features]
    try:
        models[exp] = cox_pipeline.fit(X_train_Selected, y_train_struct)
        print("Done.")
    except Exception as e:
        print(f"Err: {e}")
    finally:
        print("-"*8)

Exp: pvalue


/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:445: LinAlgWarning: Ill-conditioned matrix (rcond=8.98659e-23): result may not be accurate.
  delta = solve(optimizer.hessian, optimizer.gradient,
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:177: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:198: RuntimeWarning: overflow encountered in exp
  exp_xw = np.exp(offset + np.dot(x, w))
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:445: LinAlgWarning: Ill-conditioned matrix (rcond=5.57036e-23): result may not be accurate.
  delta = solve(optimizer.hessian, optimizer.gradient,
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:445: LinAlgWarning: Ill-condi

Err: search direction contains NaN or infinite values
--------
Exp: corr


/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:445: LinAlgWarning: Ill-conditioned matrix (rcond=3.11646e-23): result may not be accurate.
  delta = solve(optimizer.hessian, optimizer.gradient,
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:177: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:198: RuntimeWarning: overflow encountered in exp
  exp_xw = np.exp(offset + np.dot(x, w))
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:232: RuntimeWarning: invalid value encountered in add
  risk_set_x += exp_xw[k] * xk
/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:233: RuntimeWarning: invalid value encountered in add
  risk_set_xx += exp_xw[k] * xx
/home

--------


KeyboardInterrupt: 